In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd
from urllib.parse import quote


df = pd.read_json(r"C:\Users\mark.ding\big-data-ai-integration-platform\sparkVolumnTest\企業資訊_全_2024_10_24.json")

for column in df.columns:
    df[column] = df[column].apply(lambda x : ','.join(map(str,x)) if isinstance(x, list) else x)
for i in df.columns:
    df[i] = df[i].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)



In [ ]:
from sqlalchemy import create_engine, text
from sqlalchemy.pool import NullPool
import urllib.parse

# --- config ---
user = "root"
password = "!QAZ2wsx"
host = "localhost"
port = 3306
db_name = "whole_corp"

encoded_password = urllib.parse.quote_plus(password)
root_url = f"mysql+mysqlconnector://{user}:{encoded_password}@{host}:{port}/"
db_url   = f"mysql+mysqlconnector://{user}:{encoded_password}@{host}:{port}/{db_name}"

# 1) Create DB with a one-off engine (no pool, autocommit)
admin_engine = create_engine(
    root_url,
    poolclass=NullPool,                 # no pooling => no stale connections
    isolation_level="AUTOCOMMIT",       # DDL outside transactions
    pool_pre_ping=True,
)

with admin_engine.connect() as conn:
    conn.execute(text(
        f"CREATE DATABASE IF NOT EXISTS {db_name} "
        "CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci"
    ))

# important: dispose this admin engine explicitly
admin_engine.dispose()

# 2) New, clean engine pointing at the created DB
engine_db = create_engine(
    db_url,
    pool_pre_ping=True,                 # evict dead connections
)

# 3) Write the DataFrame using a brand-new connection context
#    (method='multi' + chunksize keeps it efficient)
with engine_db.connect() as conn:
    df.to_sql(
        "whole_corp",
        con=conn,
        if_exists="replace",
        index=False,
        method="multi",
        chunksize=10000,
    )


KeyboardInterrupt: 

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1512083 entries, 0 to 1512082
Data columns (total 36 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   統一編號      1512083 non-null  object
 1   公司名稱      1512083 non-null  object
 2   負責人       1344752 non-null  object
 3   登記地址      1498651 non-null  object
 4   公司狀態      862655 non-null   object
 5   資本額       1148835 non-null  object
 6   登記機關      862654 non-null   object
 7   設立日期      859885 non-null   object
 8   營業項目及代碼表  1310797 non-null  object
 9   財政營業項目    862655 non-null   object
 10  類別        862655 non-null   object
 11  縣市名稱      1498651 non-null  object
 12  區域名稱      1496807 non-null  object
 13  縣市區域      1487973 non-null  object
 14  產品        750831 non-null   object
 15  產品說明      750831 non-null   object
 16  實收資本總額    86421 non-null    object
 17  上市櫃_基本資料  1828 non-null     object
 18  官網        57726 non-null    object
 19  公司位置      71637 non-null    object
 20  聯絡人    